In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split,LeaveOneOut,KFold,RepeatedStratifiedKFold
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from collections import Counter
from functools import reduce
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
from nltk.tag.stanford import StanfordNERTagger
st = StanfordNERTagger('unigrams/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz','unigrams/stanford-ner-2018-02-27/stanford-ner-3.9.1.jar',encoding='utf-8')


In [2]:
br_to_us=pd.read_excel("Book.xlsx")
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))
decades=[2000, 1990, 1980, 1970, 1960, 1950, 1940, 1930, 1920, 1900, 1910,
            1890, 1880, 1870, 1850, 1860, 1840, 1830, 1820, 1810, 1800]
old_decades=decades[1:]
l1=dict(zip([1,-1],['Positive','Negative']))
replacements={'modifier':br_to_us_dict,'head':br_to_us_dict,'Class':l1}


In [3]:
def tupleconverter(tup):
    dicter={}
    for word,decade in tup:
        #print(word,decade)
        if word in dicter:
            dicter[word].append(decade)
        else:
            dicter[word]=[]
            dicter[word].append(decade)
            
    #for word,decades in dicter.items():
        #print(word)
        #print(decades)
        #dicter[word]=Counter(decades)
        #print(dicter[word])
    return(dicter)


def decremover(dc):

    new_constituents=[]
    for constituent,decades in dc.items():
        if len(decades)==1:
            if 2000 in decades:
                new_constituents.append(constituent)
    return(new_constituents)




def magnituder(x):
    return(np.sqrt(np.sum(np.square(x))))

def new_compound_finder(df,decade=2000):
    new_compounds=[]
    for key,val in compound_decades.items():
        if len(val)==1 and decade in val:
            new_compounds.append(key)
    return new_compounds

def lemma_maker(x):
    #print(x,y)
    return lemmatizer.lemmatize(x)

def ner_checker(stat):
    temp_list=st.tag(stat)
    if temp_list[0][1]=="PERSON" and temp_list[1][1]=="PERSON":
        return stat,True
    else:
        return stat,False
    

In [4]:
heads=pd.read_csv("heads.csv",sep="\t",header=None,usecols=[1,2,3,4])
heads.columns=["head","context","decade","count"]
#heads.info()
total_head_count=heads['count'].sum()
heads=heads.groupby(['head','context','decade'])['count'].sum().to_frame()
heads.reset_index(inplace=True)
#heads.info()
heads=heads.groupby(["head","decade"]).filter(lambda x: x['count'].sum() >= 15)
#heads.info()
heads_list=list(set(zip(heads["head"],heads["decade"])))
head_list=tupleconverter(heads_list)
new_heads=decremover(head_list)
heads=heads.loc[~heads['head'].isin(new_heads)]
#head_list=list(set(heads.index.get_level_values(0)))
heads

,head,context,decade,count
0,a_n,abridge_v,1950,1
1,a_n,abridge_v,1960,15
2,a_n,abridge_v,1970,6
3,a_n,abridge_v,1980,5
4,a_n,abridge_v,1990,16
5,a_n,abridge_v,2000,16
6,a_n,abstract_n,1990,10
7,a_n,abstract_n,2000,559
8,a_n,accredited_a,1940,25
9,a_n,accredited_a,1950,34


In [5]:
head_counts=heads.groupby(['head','decade'])['count'].sum().to_frame()
head_counts.columns=['head_count']
head_counts

head_count
head             decade            
a_n              1800            52
                 1810            40
                 1830            59
                 1840           192
                 1850           287
                 1860           266
                 1870           260
                 1880           471
                 1890           559
                 1900           966
                 1910           886
                 1920          2777
                 1930           781
                 1940          1237
                 1950          1217
                 1960          2599
                 1970          7281
                 1980          8937
                 1990         21114
                 2000         39614
aa_n             1980            92
                 1990           104
                 2000           152
aaa_n            1970            28
                 1980            92
                 1990           136
                 2000           132
aac_n            1970            24
                 1980            84
                 1990           132
...                             ...
zuichem_n        1890            24
                 1900            64
                 2000            16
zulfikar_n       1970            56
                 1980           156
                 1990           328
                 2000           460
zusammenarbeit_n 1970            60
                 1980            42
                 1990            33
zuse_n           1990            60
                 2000           102
zutphen_n        1860            68
                 1870            20
                 1880            20
                 1900            36
                 2000            20
zwienatur_n      1880            18
                 1890            35
                 1900            21
                 1930            21
                 1950            21
                 1960            42
                 1970            34
                 1980            28
                 1990            35
                 2000            28
zydom_n          1980            52
                 1990           204
                 2000           184

[175522 rows x 1 columns]

In [ ]:
head_counts.describe()

,head_count
count,175522.000
mean,1083.781
std,12875.204
min,15.000
25%,32.000
50%,76.000
75%,244.000
max,2123858.000


In [ ]:
modifiers=pd.read_csv("modifiers.csv",sep="\t",header=None,usecols=[1,2,3,4])
modifiers.columns=["modifier","context","decade","count"]
#modifiers.info()
total_modifier_count=modifiers['count'].sum()
modifiers=modifiers.groupby(['modifier','context','decade'])['count'].sum().to_frame()
modifiers.reset_index(inplace=True)
#modifiers.info()
modifiers=modifiers.groupby(["modifier","decade"]).filter(lambda x: x['count'].sum() >= 15)
#modifiers.info()
modifiers_list=list(set(zip(modifiers["modifier"],modifiers["decade"])))
modifiers_list=tupleconverter(modifiers_list)
new_modifiers=decremover(modifiers_list)
modifiers=modifiers.loc[~modifiers['modifier'].isin(new_modifiers)]
#print(new_modifiers[0:10])
#len(new_modifiers)
#modifiers.info()
modifiers

In [ ]:
modifer_counts=modifiers.groupby(['modifier','decade'])['count'].sum().to_frame()
modifer_counts.columns=['mod_count']
modifer_counts

In [ ]:
modifer_counts.describe()

In [ ]:
compounds=pd.read_csv("compounds.csv",sep="\t",header=None,usecols=[1,2,3,4,5])
compounds.columns=["modifier","head","context","decade","count"]
#compounds.info()
compounds=compounds=compounds.groupby(['modifier','head','context','decade'])['count'].sum().to_frame()
compounds.reset_index(inplace=True)
#compounds.info()
compounds=compounds.loc[~(compounds.modifier.isin(new_modifiers)&compounds['head'].isin(new_modifiers))]
compounds=compounds.groupby(["modifier","head","decade"]).filter(lambda x: x['count'].sum() >= 9.)
#compounds.info()
compounds['NER']=False
compounds['compound']=compounds['modifier'].str[:-2]+" "+compounds['head'].str[:-2]
#compounds['NER']=np.vectorize(ner_checker)(compounds['compound'])
compounds

In [ ]:
#temp_serie=
temp_series=np.vectorize(ner_checker)(compounds['compound'].unique())